# Guide to the ParameterNode

The `ParameterNode` is a container that can hold parameters, and also other `ParameterNodes`.  
In a way, one can see a ParameterNode as a dictionary, where its items can either be other dictionaries (ParameterNodes), or values (Parameters).

One example of a `ParameterNode` is an `Instrument`, though the `ParameterNode` does not need to be limited to actual instruments.

## ParameterNode with Parameters

Here we create a ParameterNode and add a Parameter to it.  
Note that we do not need to specify the parameter name, it is automatically deduced from the attribute name:

In [ ]:
from qcodes import ParameterNode, Parameter

node = ParameterNode('node')
node.p = Parameter(set_cmd=None, initial_value=1)  # Can also use node.add_parameter('p', set_cmd=None)
print('node.p.name =', node.p.name)

node.p.name = p


The `Parameter` is registered in the ParameterNode's attribute `parameters`:

In [ ]:
node.parameters

{'p': <qcodes.instrument.parameter.Parameter: p at 60981088>}

Similarly, `ParameterNode` is added as the parent of the `Parameter`:

In [ ]:
node.p.parent

ParameterNode node containing 1 parameters

This is also reflected when we get a string representation of the parameter:

In [ ]:
str(node.p)

'node_p'

**Note**: Once a `Parameter` is added to a `ParameterNode`, adding the `Parameter` to other `ParameterNodes` does not change it's parent

## ParameterNode containing ParameterNodes

A ParameterNode can also contain other ParameterNodes:

In [ ]:
node.subnode = ParameterNode()

node.subnode.p = Parameter(set_cmd=None)

str(node.subnode.p)

'node_subnode_p'

## Simplified snapshotting

When performing a measurement, a snapshot is created of all Parameters and ParameterNodes in the station.  
These snapshots can become quite messy:

In [ ]:
node.snapshot()

{'functions': {},
 'submodules': {},
 '__class__': 'qcodes.instrument.parameter_node.ParameterNode',
 'parameters': {'p': {'value': 1,
   'ts': '2018-08-30 10:03:43',
   'raw_value': 1,
   '__class__': 'qcodes.instrument.parameter.Parameter',
   'label': 'P',
   'full_name': 'node_p',
   'name': 'p'}},
 'parameter_nodes': {'subnode': {'functions': {},
   'submodules': {},
   '__class__': 'qcodes.instrument.parameter_node.ParameterNode',
   'parameters': {'p': {'value': None,
     'ts': None,
     'raw_value': None,
     '__class__': 'qcodes.instrument.parameter.Parameter',
     'label': 'P',
     'full_name': 'node_subnode_p',
     'name': 'p'}},
   'parameter_nodes': {},
   'name': 'subnode'}},
 'name': 'node'}

Depending on the type of ParameterNode, we may only be interested in the values of the parameters and nodes.  
For this reason, we can improve the readability of the snapshot by turning on `simplify_snapshot`:

In [ ]:
node.simplify_snapshot = True
node.subnode.simplify_snapshot = True

In [ ]:
node.snapshot()

{'__class__': 'qcodes.instrument.parameter_node.ParameterNode',
 'p': 1,
 'subnode': {'__class__': 'qcodes.instrument.parameter_node.ParameterNode',
  'p': None}}

We can also print a snapshot of all parameters and parameter nodes:

In [ ]:
node.print_snapshot()

node :
	parameter value
--------------------------------------------------------------------------------
p :	1 

node_subnode :
	parameter value
--------------------------------------------------------------------------------
p :	None 


## Accessing Parameters as attributes

One may have noticed that getting/setting a parameter is different from standard python classes.  
For parameters, this is done by function calls:

In [ ]:
node = ParameterNode()
node.p = Parameter(set_cmd=None)
node.p(42)  # Setting value
node.p()  # Getting value

42

Compare this to a standard python class, where the attribute behaves just like the variable it represents

In [ ]:
class C:
    pass
    
c = C()
c.p = 1
c.p

1

This can be confusing, especially if we want to convert abstract classes into `ParameterNodes` and `Parameters`.  
We would need to remember which classes are and are not ParameterNodes, and which attributes are and are not Parameters.  
As a solution to this problem, we can pass the keyword argument `use_as_atttributes`:

In [ ]:
node = ParameterNode(use_as_attributes=True)
node.p = Parameter(set_cmd=None)
node.p = 42  # Setting value
node.p  # Getting value

42

We see that now the Parameter attributes behave just like the variables they represent.  
There are cases, however, where you still want to access the Parameter object instead of its value.  
In this case, you can access the Parameter like you would in a dictionary:

In [ ]:
print(repr(node['p']))
node['p'].snapshot()

<qcodes.instrument.parameter.Parameter: p at 451256728>


{'value': 42,
 'ts': '2018-08-30 10:03:43',
 'raw_value': 42,
 '__class__': 'qcodes.instrument.parameter.Parameter',
 'label': 'P',
 'full_name': 'p',
 'name': 'p'}

**Note**: The option `use_as_attributes` can have unintended consequences, as every call to the attribute will trigger it's `get` function.  
While this is properly handled for most functions (e.g. `vars(node)`), it could cause unnecessary calls if this is not taken into account:

In [ ]:
node = ParameterNode(use_as_attributes=True)
node.p = Parameter(get_cmd=lambda: print('Get command called'))

for k in range(5):
    print(node.p)

Get command called
None
Get command called
None
Get command called
None
Get command called
None
Get command called
None


As a general rule, all Instruments should have `use_as_attributes=False` to ensure that we don't accidentally perform actions on instruments that may negatively impact the experiment.  
For more abstract ParameterNodes, especially ones where parameters don't perform ancillary get/set functions, using `use_as_attributes=True` is preferred.